## Installation

In [1]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
!pip install transformers==4.37.2 tensorflow==2.15.0 tensorflow_text==2.15.0 --no-cache-dir

In [ ]:
!pip install datasets transformers huggingface_hub

In [2]:
#check if correct versions are installed
import keras
print("Keras version:", keras.__version__)

import tensorflow as tf
print("TensorFlow version:", tf.__version__)

import transformers
print("Transformers version:", transformers.__version__)

Keras version: 2.15.0
TensorFlow version: 2.15.0
Transformers version: 4.37.2


## Imports

In [3]:
#tensorflow imports
import os
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from datasets import Dataset, DatasetDict # Import DatasetDict
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Dense, Flatten, Input, InputLayer, BatchNormalization,Dropout, LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy, categorical_crossentropy, sparse_categorical_crossentropy
from tensorflow.keras.metrics import Accuracy, TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive, files
# from datasets import load_dataset
from transformers import (BertTokenizer, BertTokenizerFast, TFBertTokenizer, RobertaTokenizerFast, DebertaTokenizerFast,
                          DataCollatorWithPadding, TFRobertaForSequenceClassification, TFRobertaForSequenceClassification,
                          TFBertModel, create_optimizer, TFDebertaForSequenceClassification,TFBertForSequenceClassification)
import pickle


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Add the directory, NOT the file itself
sys.path.append('/content/drive/MyDrive')

#import the class
from items import Item

## Data Preparation


In [6]:
#2.Load pickle files
with open('/content/drive/MyDrive/train.pkl', 'rb') as f:
    train = pickle.load(f)

with open('/content/drive/MyDrive/test.pkl', 'rb') as f:
    test = pickle.load(f)

In [7]:
#let review so datapoint in train
print(train[0].prompt)
print(train[0].price)

How much does this cost to the nearest dollar?

Star Micronics USB Thermal Receipt Printer with Device and Mfi USB Ports, Auto-cutter, and Internal Power Supply - Gray (Renewed)
High-speed printing of 43 receipts per minute with easy to setup USB connection - just Plug and Print; USB serial number feature means the PC will detect the on its Windows platform using any USB port Includes PromoPRNT promotion printing service allowing you to automatically create printed promotions in addition to receipts Drop-In and Print clamshell design allows for fast and easy paper loading; patented De-Curl function always delivers a flat receipt futurePRNT Software allows you to customize receipts to keep customers coming back time and time again with professional graphics (logos, coupons), word-triggered advertising, and more The small footprint and embedded power supply saves precious counter space Brand Star Mic

Price is $207.00
207.0


In [8]:
#let review so datapoint in train
print(train[1].prompt)
print(train[1].price)

How much does this cost to the nearest dollar?

P 27” All-in-One Desktop, AMD Ryzen 3 5300U, 8 GB RAM, 256 GB SSD, Full HD IPS Micro-Edge Display, Windows 11 Home, 4 USB Ports, Pop-up Privacy Camera, Dual Mics, Keyboard and Mouse 2021)
Get things done or lean back and relax with the HP 27” All-in-One Desktop. This computer monitor not only looks good but runs even better—sporting the latest hardware and features for the busy, modern family. Outfitted for video conferencing and collaboration, your All-in-One computer has a 1080p HP Privacy Camera with dual microphones (1) and dual front-facing speakers. Additionally, your keyboard has

Price is $650.00
649.99


In [9]:
#tester class that will evaluate any model against 250 items from the Test set
class Tester:

    def __init__(self, predictor, title=None, data=test, size=250):
        # Initialize the Tester with a prediction function, optional title, test data, and how many samples to test
        self.predictor = predictor
        self.data = data
        self.title = title or predictor.__name__.replace("_", " ").title()  # Use function name as title if not provided
        self.size = size  # Number of test data points to evaluate
        self.guesses = []  # Store model predictions
        self.truths = []   # Store actual (true) values
        self.errors = []   # Store absolute difference between prediction and truth
        self.sles = []     # Store squared log errors (for RMSLE calculation)
        self.colors = []   # Store color codes for error severity visualization

    def color_for(self, error, truth):
        # Decide color based on how big the error is (green = good, red = bad)
        if error < 40 or error / truth < 0.2:
            return "green"
        elif error < 80 or error / truth < 0.4:
            return "orange"
        else:
            return "red"

    def run_datapoint(self, i):
        # Run prediction and evaluation for a single data point at index i
        datapoint = self.data[i]
        guess = self.predictor(datapoint)         # Use model to make a prediction
        truth = datapoint.price                   # Actual price value
        error = abs(guess - truth)                # Absolute error
        log_error = math.log(truth + 1) - math.log(guess + 1)  # Log error to handle scale differences
        sle = log_error ** 2                      # Squared log error
        color = self.color_for(error, truth)      # call color_for method and assign a color based on error severity
        title = datapoint.title if len(datapoint.title) <= 40 else datapoint.title[:40] + "..."
        #append evething to the list above
        self.guesses.append(guess)
        self.truths.append(truth)
        self.errors.append(error)
        self.sles.append(sle)
        self.colors.append(color)
        # Print formatted result with color
        print(f"{COLOR_MAP[color]}{i+1}: Guess: ${guess:,.2f} Truth: ${truth:,.2f} Error: ${error:,.2f} SLE: {sle:,.2f} Item: {title}{RESET}")

    def chart(self, title):
        # Create a scatter plot to visualize how close predictions are to actual values
        max_error = max(self.errors)
        plt.figure(figsize=(12, 8))
        max_val = max(max(self.truths), max(self.guesses)) #take max of whatever in truth and guess
        plt.plot([0, max_val], [0, max_val], color='deepskyblue', lw=2, alpha=0.6)  # Diagonal reference line
        plt.scatter(self.truths, self.guesses, s=3, c=self.colors)  # Scatter plot of actual vs predicted
        plt.xlabel('Ground Truth')
        plt.ylabel('Model Estimate')
        plt.xlim(0, max_val)
        plt.ylim(0, max_val)
        plt.title(title)
        plt.show()

    def report(self):
        # Summarize model performance with average error and RMSLE
        average_error = sum(self.errors) / self.size
        rmsle = math.sqrt(sum(self.sles) / self.size)  # Root Mean Squared Log Error
        hits = sum(1 for color in self.colors if color == "green")  # Count good predictions
        title = f"{self.title} Error=${average_error:,.2f} RMSLE={rmsle:,.2f} Hits={hits/self.size*100:.1f}%"
        self.chart(title)

    def run(self):
        # Run evaluation on all data points from run_datapoint method
        self.error = 0
        for i in range(self.size):
            self.run_datapoint(i)
        self.report()

    @classmethod
    def test(cls, function):
        # Class method to quickly test a given prediction function
        cls(function).run()


In [ ]:
import pandas as pd

# 3. Convert to DataFrame
train_df = pd.DataFrame({
    'prompt': [item.prompt for item in train],
    'price': [item.price for item in train]
})

test_df = pd.DataFrame({
    'prompt': [item.prompt for item in test],
    'price': [item.price for item in test]
})

# Preview
train_df.head()

In [11]:
# 4. Convert to Hugging Face Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test": Dataset.from_pandas(test_df)
})

## Data Preparation Bert Model

In [ ]:
#Load BERT tokenizer
model_id = "bert-base-uncased"

# 5.initialize tokenizer model
tokenizer = BertTokenizerFast.from_pretrained(model_id)

In [13]:
#recheck of tokenizer is fast
tokenizer.is_fast

True

In [14]:
#6.Tokenization function
def tokenize_function(example): ##preprocess function
    return tokenizer(
        example["prompt"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

In [ ]:
#7.Tokenize entire dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [16]:
#8.Rename price → label for regression target
tokenized_dataset = tokenized_dataset.rename_column("price", "label")

In [17]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 400000
    })
    test: Dataset({
        features: ['prompt', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [18]:
# 9. Format for TensorFlow move label back
tokenized_dataset.set_format("tensorflow", columns=["input_ids", "token_type_ids", "attention_mask", "label"])

In [19]:
# 10. Define Batch
BATCH_SIZE = 32

In [20]:
tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns=["input_ids", "token_type_ids", "attention_mask", "label"],
    shuffle=True,
    batch_size=BATCH_SIZE
)

In [21]:
tf_test_dataset = tokenized_dataset["test"].to_tf_dataset(
    columns=["input_ids", "token_type_ids", "attention_mask", "label"],
    shuffle=False,
    batch_size=BATCH_SIZE
)

In [22]:
#verfy all is well
for i in tf_train_dataset.take(1):
  print(i)

{'label': <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([127.96, 413.66, 165.99, 142.61,  27.99, 310.  , 135.  , 249.99,
       428.  , 159.  , 324.5 , 999.  , 289.89, 440.  , 317.95,  36.99,
       254.99, 344.98, 146.25, 148.68, 189.  , 399.  , 134.88, 275.76,
       383.75, 699.  , 142.45,  73.99, 203.99,  78.95,  37.69, 234.99],
      dtype=float32)>, 'input_ids': <tf.Tensor: shape=(32, 128), dtype=int64, numpy=
array([[  101,  2129,  2172, ...,  1011,  4621,   102],
       [  101,  2129,  2172, ...,  5691,  1010,   102],
       [  101,  2129,  2172, ..., 14142, 24096,   102],
       ...,
       [  101,  2129,  2172, ...,  1020,  4206,   102],
       [  101,  2129,  2172, ...,  2030,  5010,   102],
       [  101,  2129,  2172, ...,  9859, 13938,   102]])>, 'token_type_ids': <tf.Tensor: shape=(32, 128), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0

In [23]:
# we will push labels back as it is outputs
def swap_postions(dataset):
  return{'input_ids':dataset['input_ids'],
         'token_type_ids':dataset['token_type_ids'],
         'attention_mask':dataset['attention_mask'],},dataset['label']

In [24]:
tf_train_dataset = tf_train_dataset.map(swap_postions).prefetch(tf.data.AUTOTUNE)
tf_val_dataset = tf_test_dataset.map(swap_postions).prefetch(tf.data.AUTOTUNE)

In [25]:
# recheck all is well
for i in tf_train_dataset.take(1):
  print(i)

({'input_ids': <tf.Tensor: shape=(32, 128), dtype=int64, numpy=
array([[  101,  2129,  2172, ...,  5061,  1012,   102],
       [  101,  2129,  2172, ...,  2813,  7766,   102],
       [  101,  2129,  2172, ...,     0,     0,     0],
       ...,
       [  101,  2129,  2172, ..., 14101,  2640,   102],
       [  101,  2129,  2172, ...,  2094,  5110,   102],
       [  101,  2129,  2172, ...,  6519, 10312,   102]])>, 'token_type_ids': <tf.Tensor: shape=(32, 128), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(32, 128), dtype=int64, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])>}, <tf.Tensor: shape=(32,), dtype=float

In [26]:
tf_train_dataset

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [27]:
tf_val_dataset

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

## Data Preparation for Roberta Model

In [ ]:
#load the roberta tokenizer
model_id="roberta-base"

#instantiate the model
tokenizer=RobertaTokenizerFast.from_pretrained(model_id)

In [32]:
#6.Tokenization function
def tokenize_function(example): ##preprocess function
    return tokenizer(
        example["prompt"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

In [ ]:
#7.Tokenize entire dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [36]:
tokenized_dataset['train'][0]

{'prompt': 'How much does this cost to the nearest dollar?\n\nStar Micronics USB Thermal Receipt Printer with Device and Mfi USB Ports, Auto-cutter, and Internal Power Supply - Gray (Renewed)\nHigh-speed printing of 43 receipts per minute with easy to setup USB connection - just Plug and Print; USB serial number feature means the PC will detect the on its Windows platform using any USB port Includes PromoPRNT promotion printing service allowing you to automatically create printed promotions in addition to receipts Drop-In and Print clamshell design allows for fast and easy paper loading; patented De-Curl function always delivers a flat receipt futurePRNT Software allows you to customize receipts to keep customers coming back time and time again with professional graphics (logos, coupons), word-triggered advertising, and more The small footprint and embedded power supply saves precious counter space Brand Star Mic\n\nPrice is $207.00',
 'label': 207.0,
 'input_ids': [0,
  6179,
  203,
 

In [37]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'label', 'input_ids', 'attention_mask'],
        num_rows: 400000
    })
    test: Dataset({
        features: ['prompt', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [39]:
tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols="label",
    shuffle=True,
    batch_size=32
)

In [40]:
tf_val_dataset = tokenized_dataset["test"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols="label",
    shuffle=False,
    batch_size=32
)

In [41]:
tf_train_dataset

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [42]:
tf_val_dataset

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [45]:
# we will push labels back as it is outputs
def swap_positions(features, label):
  return{'input_ids':features['input_ids'],
         'attention_mask':features['attention_mask'],},label

In [46]:
tf_train_dataset=tf_train_dataset.map(swap_positions).prefetch(tf.data.AUTOTUNE)
tf_val_dataset=tf_val_dataset.map(swap_positions).prefetch(tf.data.AUTOTUNE)

In [47]:
for i in tf_train_dataset.take(1):
  print(i)

({'input_ids': <tf.Tensor: shape=(32, 128), dtype=int64, numpy=
array([[    0,  6179,   203, ...,     4,  1244,     2],
       [    0,  6179,   203, ...,  2444,     6,     2],
       [    0,  6179,   203, ...,   465,    41,     2],
       ...,
       [    0,  6179,   203, ..., 27038,     5,     2],
       [    0,  6179,   203, ..., 36676,  9785,     2],
       [    0,  6179,   203, ..., 21316,  7670,     2]])>, 'attention_mask': <tf.Tensor: shape=(32, 128), dtype=int64, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])>}, <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([572.25,  22.64,  27.99, 191.71, 112.27, 488.89, 155.99, 185.28,
       245.  , 310.99, 332.  , 195.32, 499.  , 150.79,  74.99, 639.  ,
         9.34,   2.99, 114.99,  43.99, 461.24, 288.  ,  16.  ,  39.99,
       125.99, 149.  , 167.37,  77.85, 189.45,  15

## Data Preparation of Deberta

In [ ]:
#import the model from hugging face
model_id = "microsoft/deberta-base"

#initialize the tokenizer model
tokenizer=DebertaTokenizerFast.from_pretrained(model_id)

In [52]:
def tokenize_function(example):
    return tokenizer(
        example["prompt"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [54]:
#review data
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'price', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 400000
    })
    test: Dataset({
        features: ['prompt', 'price', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [55]:
#review one data element
tokenized_dataset['train'][0]

{'prompt': 'How much does this cost to the nearest dollar?\n\nStar Micronics USB Thermal Receipt Printer with Device and Mfi USB Ports, Auto-cutter, and Internal Power Supply - Gray (Renewed)\nHigh-speed printing of 43 receipts per minute with easy to setup USB connection - just Plug and Print; USB serial number feature means the PC will detect the on its Windows platform using any USB port Includes PromoPRNT promotion printing service allowing you to automatically create printed promotions in addition to receipts Drop-In and Print clamshell design allows for fast and easy paper loading; patented De-Curl function always delivers a flat receipt futurePRNT Software allows you to customize receipts to keep customers coming back time and time again with professional graphics (logos, coupons), word-triggered advertising, and more The small footprint and embedded power supply saves precious counter space Brand Star Mic\n\nPrice is $207.00',
 'price': 207.0,
 'input_ids': [1,
  6179,
  203,
 

In [56]:
tokenized_dataset = tokenized_dataset.rename_column("price", "label")

In [57]:
tokenized_dataset.set_format("tensorflow", columns=["input_ids", "attention_mask", "label"])

In [63]:
#convert tokenized dataset to tensorflow
tf_dataset = tokenized_dataset['train'].to_tf_dataset(
    columns = ['input_ids','attention_mask', 'label'],
    shuffle= True,
    batch_size= BATCH_SIZE,
)

In [64]:
#swap the postions so o/p intent is after inputs
def swap_postions(dataset):
  return {'input_ids':dataset['input_ids'],
          'attention_mask':dataset['attention_mask'],
          }, dataset['label']

In [65]:
#call the function
tf_dataset = tf_dataset.map(swap_postions)

In [66]:
# split train and test dataset
train_dataset = tf_dataset.take(int(0.9*len(tf_dataset)))
val_dataset = tf_dataset.skip(int(0.9*len(tf_dataset)))

In [67]:
#review data
for i in val_dataset.take(1):
  print(i)

({'input_ids': <tf.Tensor: shape=(32, 128), dtype=int64, numpy=
array([[    1,  6179,   203, ..., 14950,  1757,     2],
       [    1,  6179,   203, ...,     6, 17175,     2],
       [    1,  6179,   203, ...,  3139,  9620,     2],
       ...,
       [    1,  6179,   203, ..., 13918,    31,     2],
       [    1,  6179,   203, ...,   146,    13,     2],
       [    1,  6179,   203, ...,     4,   134,     2]])>, 'attention_mask': <tf.Tensor: shape=(32, 128), dtype=int64, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])>}, <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([174.99, 129.95, 139.  ,  53.99, 305.98,  37.99, 130.92,  99.99,
       256.09, 118.3 , 188.15, 233.8 , 409.99, 203.69,  75.9 , 168.98,
       224.99,  18.99, 190.  , 500.  , 185.2 , 739.99, 549.  ,  15.99,
       148.99, 799.  , 225.95, 289.99, 171.99, 299

# Modeling

## Based on TFBertForSequenceClassification

In [ ]:
#Instantiate the model
#Load prebuilt BERT model for regression
model = TFBertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=1  # for regression
)

model.summary()

## Based on TFRobertaForSequenceClassification

In [ ]:
#instantiate the model
model=TFRobertaForSequenceClassification.from_pretrained(model_id,num_labels=2)
model.summary()

## Based on TFDebertaForSequenceClassification

In [69]:
model = TFDebertaForSequenceClassification.from_pretrained(model_id, num_labels=1)
model.summary()

All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

Some layers of TFDebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['cls_dropout', 'pooler', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_deberta_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 deberta (TFDebertaMainLaye  multiple                  138601728 
 r)                                                              
                                                                 
 pooler (TFDebertaContextPo  multiple                  590592    
 oler)                                                           
                                                                 
 cls_dropout (TFDebertaStab  multiple                  0         
 leDropout)                                                      
                                                                 
 classifier (Dense)          multiple                  769       
                                                                 
Total params: 139193089 (530.98 MB)
Trainable params: 139193089 (530.98 MB)
Non-trainable pa

#Model Training

In [70]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss='mse',
    metrics=['mae']
)

In [71]:
history = model.fit(
    tf_train_dataset,
    validation_data=tf_val_dataset,
    epochs=2
)

Epoch 1/2


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


    5/12500 [..............................] - ETA: 21:02:34 - loss: 91461.3594 - mae: 235.4767

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('model_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Testing

In [ ]:
inputs = tokenizer(["this movie looks very interesting, i love the fact that the actors do a great job in showing how people lived in the 18th century, which wasn't very good at all. But atleast this movie recreates this scenes! ",
                    "very good start, but movie started becoming uninteresting at some point though initially i thought it would have been much more fun. There was too much background noise, but later on towards the middle of the movie, my favorite character got in and he did a great job, so over "], padding=True,return_tensors="tf")

logits = model(**inputs).logits
print(logits)